In [1]:
from illufly.llm import MemoryTopic, MemoryQA

In [2]:
a = MemoryTopic(topic_text="hi")
a

MemoryTopic(user_id=None, created_at=1743300829.801975, topic_text='hi', topic_text_hash='8f434346648f6b96df89dda901c5176b10a6d83961dd3c1ac88b59b2dc327aa4')

In [3]:
a.model_dump()

{'user_id': None,
 'created_at': 1743300829.801975,
 'topic_text': 'hi',
 'topic_text_hash': '8f434346648f6b96df89dda901c5176b10a6d83961dd3c1ac88b59b2dc327aa4'}

In [4]:
a.topic_text="abc"

In [5]:
a.model_dump()

{'user_id': None,
 'created_at': 1743300664.49215,
 'topic_text': 'abc',
 'topic_text_hash': 'ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad'}

In [9]:
import pandas as pd
import re
import html

def safe_extract_markdown_tables(md_text):
    """安全提取Markdown表格为结构化数据（支持多表）"""
    tables = []
    # 匹配所有表格（非贪婪模式）
    table_blocks = re.finditer(
        r'\|(.+?)\|\n\|([\-: ]+\|)+\n((?:\|.*\|\n?)+)',
        md_text,
        re.DOTALL
    )
    
    for table in table_blocks:
        try:
            headers = [h.strip() for h in table.group(1).split('|') if h.strip()]
            rows = []
            
            for row in table.group(3).split('\n'):
                if not row.strip():
                    continue
                cells = [
                    html.unescape(cell).strip() 
                    for cell in row.split('|')[1:-1]
                ]
                if len(cells) == len(headers):
                    rows.append(cells)
            
            if headers and rows:
                tables.append(pd.DataFrame(rows, columns=headers))
        except Exception as e:
            print(f"表格解析失败: {e}")
    
    return tables

# 测试用例
md_table = """
```markdown
| 主题           | 问题                     | 答案                  |
|---------------|-------------------------|----------------------|
| [计算]比较策略 | 用户对数字比较的要求是？ | 按小数点后逐位比较「记住这个计算策略」 |
| [UI]按钮颜色  | 用户偏好什么色调？       | 蓝色系，透明度≤50%     |
```
"""

tables = safe_extract_markdown_tables(md_table)
tables
# # 使用示例
# tables = safe_extract_markdown_tables(md_table)
# for df in tables:
#     print(df.to_dict('records'))

[         主题            问题                   答案
 0  [计算]比较策略  用户对数字比较的要求是？  按小数点后逐位比较「记住这个计算策略」
 1  [UI]按钮颜色     用户偏好什么色调？          蓝色系，透明度≤50%]

In [14]:
tables[0].to_dict()

{'主题': {0: '[计算]比较策略', 1: '[UI]按钮颜色'},
 '问题': {0: '用户对数字比较的要求是？', 1: '用户偏好什么色调？'},
 '答案': {0: '按小数点后逐位比较「记住这个计算策略」', 1: '蓝色系，透明度≤50%'}}